범주형 데이터 장르를 이용한 테스트

In [45]:
import re
import requests
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')

ps4를 통한 예제

In [46]:
df_genres=pd.read_csv("../data/ps4.csv")
df_genres.dtypes

genres           object
metascore         int64
developer        object
userscore       float64
rating           object
publisher        object
title            object
summary          object
release_date     object
dtype: object

장르들은 칼럼하나에 여러개 속해있음
장르들을 원핫인코딩으로 변환

In [55]:
dic_genre={}
for idx,row in df_genres.iterrows():
    nowg=row['genres']
    nowg=nowg.split('|')
    nowg.pop()
    for x in nowg:
        if(x in dic_genre):
            dic_genre[x]+=1
        else:
            dic_genre[x]=0
lgenres=list(dic_genre.keys())
jgenres = pd.DataFrame(np.zeros((len(df_genres), len(lgenres))),columns=lgenres)
for i, genre in enumerate(df_genres.genres):
    k=genre.split('|')
    k.pop()
    k=list(set(k))
    jgenres.loc[i, k] = 1
df_game=df_genres[['metascore','userscore']]
df_game_t = df_game.join(jgenres.add_prefix('genre_'))

developer를 원핫인코딩으로 변환 메타스코어를 예측할 예정이므로 userscore을 스케일링

In [56]:
print(df_game_t)
#df_game_t=pd.get_dummies(df_game_t,columns=['developer'])

      metascore  userscore  genre_Simulation  genre_Virtual  genre_Career  \
0            43        4.4               1.0            1.0           1.0   
1            43        4.2               0.0            0.0           0.0   
2            43        2.6               0.0            0.0           0.0   
3            43        1.6               0.0            0.0           0.0   
4            42        5.6               0.0            0.0           0.0   
...         ...        ...               ...            ...           ...   
2633         87        7.3               0.0            0.0           0.0   
2634         87        8.2               0.0            0.0           0.0   
2635         87        7.7               0.0            0.0           0.0   
2636         87        7.8               0.0            0.0           0.0   
2637         87        5.2               0.0            0.0           0.0   

      genre_Action Adventure  genre_Sci-Fi  genre_General  genre_Arcade  \


훈련용 x축 y축 추출

In [57]:
scaler = StandardScaler()
data_features = df_game_t.drop("metascore", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(data_features, df_game_t['metascore'], random_state = 0)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape)
pd.DataFrame(X_train)

(1978, 131)


,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,-1.388227,-0.240321,-0.093108,-0.087415,1.949608,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
1,0.043341,4.161107,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
2,-0.785461,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
3,1.098180,-0.240321,-0.093108,-0.087415,1.949608,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
4,-0.182696,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,1.560137,2.924988,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,0.872143,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
1974,0.420069,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
1975,-0.001867,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,4.024922,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249
1976,0.495415,-0.240321,-0.093108,-0.087415,-0.512924,-0.108465,-0.640969,-0.341882,-0.248452,-0.078127,...,-0.031814,-0.031814,-0.031814,-0.02249,-0.02249,-0.02249,-0.031814,-0.02249,-0.02249,-0.02249


로지스틱 회귀모델 적용 

In [58]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print("테스트 점수 : {:.2f}".format(logreg.score(X_test, y_test)))

테스트 점수 : 0.05


상관계수 조사

In [54]:
import scipy.stats as stats
U=df_game_t['userscore'].values
M=df_game_t['metascore'].values
print("correl: {:.2f}".format(stats.pearsonr(U,M)[0]))
print("pval: {:.4f}".format(stats.pearsonr(U,M)[1]))

correl: 0.52
pval: 0.0000
